In [ ]:
import math
import numpy as np
from sklearn import datasets

In [ ]:
# calculate entropy of a node in decision tree
def entropy(p1, n1):
    if p1==0 and n1==0:
        return 1
    elif p1==0 or n1==0:
        return 0
    
    pp = p1 / (p1+n1)
    np = n1 / (p1+n1)
    return -pp*math.log2(pp) - np*math.log2(np)

In [ ]:
# calculate information gain of a factor
def ig(p1, n1, p2, n2):
    num = p1+n1+p2+n2
    num1 = p1+n1
    num2 = p2+n2
    
    parent_entropy = entropy(p1+p2, n1+n2)
    children_wavg_entropy = (num1/num)*entropy(p1, n1) +(num2/num)*entropy(p2, n2)
    return parent_entropy - children_wavg_entropy

In [ ]:
def build_tree(feature, target):

    node = dict()
    node["data"] = list(range(len(target)))
    tree = []
    tree.append(node)
    t = 0

    while t<len(tree):
        # print("Current t: ", t)
        current_node_data = tree[t]["data"]

        # all data does not play tennis
        if sum(target[current_node_data]) == 0:
            tree[t]["is_leaf"] = True
            tree[t]["decision"] = "not play"

        # all data play tennis
        elif sum(target[current_node_data]) == len(current_node_data):
            tree[t]["is_leaf"] = True
            tree[t]["decision"] = "play"

        else:
            best_ig = 0

            # iterate every feature
            for i in range(feature.shape[1]):
                all_values = list(set(feature[current_node_data, i]))
                all_values.sort()

                # iterate every partition, making a binary tree
                # for example, values=1, 2, 3 => (1, 2&3) or (1&2, 3)
                for j in range(len(all_values)-1):
                    threshold = (all_values[j] + all_values[j+1])/2

                    group1 = []
                    group2 = []

                    for k in current_node_data:
                        if feature[k, i]<= threshold:
                            group1.append(k)
                        else:
                            group2.append(k)


                    group1_play = sum(target[group1] == 1)
                    group1_nplay = sum(target[group1] == 0)
                    group2_play = sum(target[group2] == 1)
                    group2_nplay = sum(target[group2] == 0)

                    current_ig = ig(group1_play, group1_nplay, group2_play, group2_nplay)

                    if current_ig > best_ig:
                        best_ig = current_ig
                        best_group1 = group1
                        best_group2 = group2
                        best_feature = i
                        best_threshold = threshold

            if best_ig > 0:

                # create new two children
                left_child = dict()
                left_child["data"] = best_group1
                tree.append(left_child)
                tree[t]["child_left"] = len(tree)-1

                right_child = dict()
                right_child["data"] = best_group2
                tree.append(right_child)
                tree[t]["child_right"] = len(tree)-1

                # update parent node
                tree[t]["feature"] = best_feature
                tree[t]["threshold"] = best_threshold
                tree[t]["is_leaf"] = False



            else:
                tree[t]["is_leaf"] = True
                if sum(target[current_node_data]==1) > sum(target[current_node_data]==0):
                    tree[t]["decision"] = "play"
                else:
                    tree[t]["decision"] = "not play"

        t += 1
    
    return tree

In [ ]:
def testing(tree, feature, target, iteration):
    
    print("#%.2d Cross Validation (incorrect sample/total sample):" %(iteration+1), end=" ")
    
    incorrect = 0

    for i in range(len(target)):
        test_feature = feature[i, :]
        now = 0
        while tree[now]["is_leaf"] == False:
            classifier = tree[now]["feature"]
            threshold = tree[now]["threshold"]

            if test_feature[classifier] <= threshold:
                now = tree[now]["child_left"]
            else:
                now = tree[now]["child_right"]

        if target[i] == 0:
            true_ = "not play"
        else:
            true_ = "play"

        # print("#{} sample: True={}, Pred={}".format(str(i+1), true_, tree[now]["decision"]))
        
        if true_ != tree[now]["decision"]:
            incorrect += 1
    
    print("{}/{}".format(str(incorrect), str(len(target))))

In [ ]:
def load_dataset():
    iris_dataset = datasets.load_iris()
    feature = iris_dataset["data"]
    target = iris_dataset["target"]

    # remove data with target = 2
    mask = target != 2
    feature = feature[mask]
    target = target[mask]
    
    # reshape target to concatenate it as new column with feature
    data = np.concatenate((feature, target.reshape((-1, 1))), axis=1)
    
    # shuffle 2d array
    np.random.shuffle(data)
    
    return data

In [ ]:
if __name__ == "__main__":
    
    data = load_dataset()
    
    iteration = 0
    for i in range(10, len(data)+10, 10):
        # in mask, 1 means this data is for training, 0 means this data is for testing
        mask = np.ones((len(data)))
        mask[i-10:i] = 0
        
        training_data = data[mask==1]
        testing_data = data[mask==0]
        
        tree = build_tree(training_data[:, :-1], training_data[:, -1])
        testing(tree, testing_data[:, :-1], testing_data[:, -1], iteration)
        
        iteration += 1